<center><h1>Procesamiento y transformación de variables (características)</h1></center>

## 1. Introducción.

Para entender cómo funciona la regresión lineal, nos limitamos a usar características del conjunto de datos de entrenamiento que no contenían valores faltantes y que ya estaban en una representación numérica conveniente. En esta lección, exploraremos cómo transformar algunas de las características restantes para poder usarlas en nuestro modelo. En términos generales, el proceso de procesamiento y creación de nuevas características se conoce como ingeniería de características (**feature engineering**). La ingeniería de características o variables es un poco un arte y tener conocimiento en el dominio específico (en este caso, bienes raíces) puede ayudarlo a crear mejores características. En esta lección, nos centraremos en algunas estrategias independientes del dominio que funcionan para todos los problemas.

En la primera mitad de esta lección, nos centraremos solo en las columnas que no contienen valores perdidos pero que aún no tienen el formato adecuado para usar en un modelo de regresión lineal. En la segunda mitad de esta lección, exploraremos algunas formas de lidiar con los valores faltantes.

Entre las columnas que no contienen valores faltantes, algunos de los problemas comunes incluyen:

- la columna no es numérica (por ejemplo, un código de zonificación representado mediante texto)
- la columna es numérica pero no ordinal (por ejemplo, valores de código postal)
- la columna es numérica pero no es representativa del tipo de relación con la columna de destino (por ejemplo, valores de año)

Comencemos por filtrar el conjunto de entrenamiento a solo las columnas que no contienen valores faltantes.

### Ejercicio
- Seleccione solo las columnas del dataframe de `train` que no contengan valores faltantes.
- Asigne el marco de datos resultante, que contiene solo estas columnas, a `df_no_mv`.
- Explore las variables para familiarizarse con estas columnas.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('AmesHousing.txt', delimiter='\t')
train = data[:1460].copy()
test = data[1460:].copy()

In [3]:
not_null_colls = train.columns[train.notnull().all()].values.tolist()
df_no_mv = train[not_null_colls]

In [4]:
df_no_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            1460 non-null   int64  
 1   PID              1460 non-null   int64  
 2   MS SubClass      1460 non-null   int64  
 3   MS Zoning        1460 non-null   object 
 4   Lot Area         1460 non-null   int64  
 5   Street           1460 non-null   object 
 6   Lot Shape        1460 non-null   object 
 7   Land Contour     1460 non-null   object 
 8   Utilities        1460 non-null   object 
 9   Lot Config       1460 non-null   object 
 10  Land Slope       1460 non-null   object 
 11  Neighborhood     1460 non-null   object 
 12  Condition 1      1460 non-null   object 
 13  Condition 2      1460 non-null   object 
 14  Bldg Type        1460 non-null   object 
 15  House Style      1460 non-null   object 
 16  Overall Qual     1460 non-null   int64  
 17  Overall Cond  

## 2. Variables categóricas.
Notará que algunas de las columnas en el marco de datos `df_no_mv` contienen valores de cadena. Si estas columnas contienen solo un conjunto limitado de valores únicos, se conocen como  variables(características) categóricas. Como sugiere el nombre, una característica categórica agrupa una instancia en una categoría específica. Estos son algunos ejemplos del conjunto de datos:

In [5]:
print(train['Utilities'].value_counts())

AllPub    1457
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64


In [7]:
print(train['Street'].value_counts(), '\n')

print(train['House Style'].value_counts())

Pave    1455
Grvl       5
Name: Street, dtype: int64 

1Story    743
2Story    440
1.5Fin    160
SLvl       60
SFoyer     35
2.5Unf     11
1.5Unf      8
2.5Fin      3
Name: House Style, dtype: int64


Para usar estas características en nuestro modelo, necesitamos transformarlas en representaciones numéricas. Afortunadamente, pandas lo hace fácil porque la biblioteca tiene un tipo de [datos categórico](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html). Podemos convertir cualquier columna que no contenga valores faltantes (o se generará un error) al tipo de datos categóricos usando el método [`pandas.Series.astype()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html):

In [9]:
print(train['Utilities'].astype('category').dtype)
print(train['Utilities'].dtype)

category
object


In [10]:
train['Utilities'] = train['Utilities'].astype('category')

Cuando una columna se convierte a tipo de datos categóricos, pandas asigna un código a cada valor único en la columna. A menos que accedamos a estos valores directamente, la mayoría de las operaciones de manipulación de pandas que funcionan para las columnas de cadenas (texto) también funcionarán para las categóricas.

In [11]:
print(train['Utilities'])

0       AllPub
1       AllPub
2       AllPub
3       AllPub
4       AllPub
         ...  
1455    AllPub
1456    AllPub
1457    AllPub
1458    AllPub
1459    AllPub
Name: Utilities, Length: 1460, dtype: category
Categories (3, object): ['AllPub', 'NoSeWa', 'NoSewr']


Necesitamos usar el descriptor de acceso `.cat` seguido de la propiedad `.codes` para acceder realmente a la representación numérica subyacente de una columna:

In [12]:
print(train['Utilities'].cat.codes)

0       0
1       0
2       0
3       0
4       0
       ..
1455    0
1456    0
1457    0
1458    0
1459    0
Length: 1460, dtype: int8


### Ejercicio
- Convierta todas las columnas de texto en `train` al tipo de datos categóricos. Para seleccionar columnas de un tipo de dato en específico puede usar [`pandas.DataFrame.select_dtypes`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html)
- Seleccione la columna `Utilities`, devuelva los códigos categóricos y muestre los recuentos de valores únicos para esos códigos: `train['Utilities'].cat.codes.value_counts()` 

In [18]:
print(df_no_mv.select_dtypes(include=['object']).columns)
text_cols = df_no_mv.select_dtypes(include=['object']).columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Exter Cond',
       'Foundation', 'Heating', 'Heating QC', 'Central Air', 'Electrical',
       'Kitchen Qual', 'Functional', 'Paved Drive', 'Sale Type',
       'Sale Condition'],
      dtype='object')


In [24]:
for col in text_cols:
    print(f'{col}:{len(train[col].unique())}')
    train[col] = train[col].astype('category')

MS Zoning:6
Street:2
Lot Shape:4
Land Contour:4
Utilities:3
Lot Config:5
Land Slope:3
Neighborhood:26
Condition 1:9
Condition 2:6
Bldg Type:5
House Style:8
Roof Style:6
Roof Matl:5
Exterior 1st:14
Exterior 2nd:16
Exter Qual:4
Exter Cond:5
Foundation:6
Heating:6
Heating QC:4
Central Air:2
Electrical:4
Kitchen Qual:5
Functional:7
Paved Drive:3
Sale Type:9
Sale Condition:5


In [26]:
print(train['Utilities'].cat.codes.value_counts())

0    1457
2       2
1       1
dtype: int64


## 3. Codificación ficticia.
Cuando convertimos una columna datos categóricos, pandas asigna un número `0` a `n-1` (donde `n` es el número de valores únicos en una columna) para cada valor. El inconveniente de este enfoque es que aquí se viola uno de los supuestos de la regresión lineal. La regresión lineal opera bajo el supuesto de que las características están linealmente correlacionadas con la columna de destino. Sin embargo, para una característica categórica, no hay un significado numérico real para los códigos categóricos que pandas asignó para esa columna. Un aumento en la columna Utilidades de 1 a 2 no tiene un valor de correlación con la columna de destino y, en cambio, los códigos categóricos se utilizan para la unicidad y exclusividad (la categoría asociada con 0 es diferente a la asociada con 1).

La solución común es usar una técnica llamada [codificación ficticia](https://en.wikipedia.org/wiki/Dummy_variable_%28statistics%29). En lugar de tener una sola columna con `n` códigos enteros, tenemos `n` columnas binarias. Así es como se vería para la columna Utilidades:

| Utilities_AllPub | Utilities_NoSewr | Utilities_NoSeWa |
|------------------|------------------|------------------|
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |
| 1                | 0                | 0                |

Debido a que los valores originales para las primeras 4 filas eran AllPub, en el nuevo esquema contienen el valor binario para verdadero (1) en la columna Utilities_AllPub y 0 para las otras 2 columnas.

Afortunadamente, Pandas tiene una función conveniente para ayudarnos a aplicar esta transformación para todas las columnas de texto llamada pandas.get_dummies().

### Ejercicio

## 4. Transformación de variables numéricas impropias.
### Ejercicio

## 5. Valores faltantes.
### Ejercicio

## 6. Imputación de valores faltantes.
### Ejercicio

## 7. Próximos pasos.
